## List the capacities and their most recent recorded status

In [ ]:
%%sql
select * from dbo.capacity_list
where EffectiveDate > dateadd(current_timestamp(), -3)
order by CapacityName, EffectiveDate

## List the most recent items changed

In [ ]:
%%sql
--changes from the last day, note does not include items where workspace is deleted
select c.CapacityName, w.WorkspaceName, i.* 
from dbo.workspace_item_list i
    join workspace_list w on i.WorkspaceId = w.WorkspaceId and w.current_row = True
    join capacity_list c on w.CapacityId =c.CapacityId and c.current_row = True
where ItemId in (
    SELECT ItemId FROM dbo.workspace_item_list 
    where current_row = false and EndDate > date_add(current_timestamp,-1)
    )
order by ItemId, EffectiveDate

## List the Capacities and related Workspaces

In [ ]:
%%sql
--show workspaces by capacity
select 
    c.CapacityId, c.CapacityName, c.Region
    , w.WorkspaceId, w.WorkspaceName
from
dbo.capacity_list c
left join dbo.workspace_list w on c.CapacityId = w.CapacityId and c.current_row = True and w.current_row = True
order by 
    CapacityName, WorkspaceName


## Summarise the number of workspaces and items on each capacity

In [ ]:
%%sql
--
select c.CapacityName, w.WorkspaceName, i.Type as ItemType, count(ItemId) as ItemCount
from dbo.workspace_item_list i
    join workspace_list w on i.WorkspaceId = w.WorkspaceId and w.current_row = True
    join capacity_list c on w.CapacityId =c.CapacityId and c.current_row = True
where 
    i.current_row= True
group by 
    c.CapacityName, w.WorkspaceName, i.Type 

In [ ]:
%%sql
--
select c.CapacityName, w.WorkspaceName, count(ItemId) as ItemCount
from dbo.workspace_item_list i
    join workspace_list w on i.WorkspaceId = w.WorkspaceId and w.current_row = True
    join capacity_list c on w.CapacityId =c.CapacityId and c.current_row = True
where 
    i.current_row= True
group by 
    c.CapacityName, w.WorkspaceName

In [ ]:
%%sql
select i.Type as ItemType, count(ItemId) as ItemCount
from dbo.workspace_item_list i
where 
    i.current_row= True
group by 
    i.Type 

## Pivot some of the Item Types to summarise Workspaces

In [ ]:
%%sql
--
select * from
(select c.CapacityName, w.WorkspaceName, i.Type as ItemType, ItemId
from dbo.workspace_item_list i
    join workspace_list w on i.WorkspaceId = w.WorkspaceId and w.current_row = True
    join capacity_list c on w.CapacityId =c.CapacityId and c.current_row = True
where 
    i.current_row= True
) md
    PIVOT
        (count(ItemId) for ItemType IN ('SemanticModel' AS SemanticModel, 'Lakehouse' AS Lakehouse
            , 'Notebook' AS Notebook, 'SQLEndpoint' AS SQLEndpoint, 'DataPipeline' AS DataPipeline)
)